# 📊 Analyse des données

## 📌 Introduction
Ce notebook a pour objectif d'analyser les données afin d'en extraire des insights pertinents.


In [1]:
from utils import *

## 📂 Importation des données

In [2]:
df = load_cleaned_data()

## 📊 Visualisation des données

### Détermination de la Hauteur Optimale de la Digue

 🎯 Objectif : Nous cherchons à déterminer la hauteur optimale de la digue en analysant la distribution des hauteurs de crues.

 📌 Méthodologie : Calculer le **percentile 95%** des hauteurs de crues.

 💡 Interprétation : Une digue dimensionnée selon le **percentile 95%** couvre 95% des crues historiques. **5% des crues** risquent encore de dépasser cette hauteur, mais cela réduit considérablement les coûts de construction.



In [3]:
visualize_distribution(df)

#---hauteur optimale----
hauteur_opt = get_optimal_digue_height(df)
print(f"Hauteur optimale de la digue : {hauteur_opt:.2f} m")

#---seuil optimal---
visualize_distribution_with_threshold(df, hauteur_opt)

Hauteur optimale de la digue : 6.17 m


In [4]:
visualize_boxplot(df)
visualize_time_series(df)